## **1. Install Selenium**

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list << "EOF"
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << "EOF"
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.d4yFe7GwlL/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.QjCPvca7EG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.cuhXVCjX4A/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu 

## **2. Import libraries**

In [ ]:
import os
import requests
import time
import json

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## **3. Crawl audio**

In [ ]:
WEBDRIVER_DELAY_TIME_INT = 10
TIMEOUT_INT = 10
service = Service(executable_path=r"/usr/bin/chromedriver")
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("window-size=1920x1080")
chrome_options.headless = True
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.implicitly_wait(TIMEOUT_INT)
wait = WebDriverWait(driver, WEBDRIVER_DELAY_TIME_INT)

In [ ]:
def extract_audio_links_from_menu(menu_url, driver):
    driver.get(menu_url)
    container = wait.until(EC.presence_of_element_located(
        (By.CSS_SELECTOR, "div.w-full.flex.flex-col.gap-3.pt-3")
    ))
    play_items = container.find_elements(By.CSS_SELECTOR, "div.play-item")
    links = []
    for item in play_items:
        try:
            a_tag = item.find_element(By.CSS_SELECTOR, ".ptxt-track a")
            link = a_tag.get_attribute("href")
            links.append(link)
        except Exception:
            continue
    return links

In [ ]:
def extract_track_info(driver):
    audio_div = WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div[data-track-info]"))
    )
    return json.loads(audio_div.get_attribute("data-track-info"))

def extract_genres(driver):
    try:
        genre_elem = driver.find_element(By.CSS_SELECTOR, "span.md\\:col-span-6.flex.flex-wrap.gap-3")
        return [a.text.strip() for a in genre_elem.find_elements(By.TAG_NAME, "a") if a.text.strip()]
    except Exception:
        return []

def extract_duration(driver):
    try:
        duration_elem = driver.find_element(By.CSS_SELECTOR, "span.w-12.ml-auto.md\\:ml-0.col-span-2.inline-flex.justify-end.items-center")
        return duration_elem.text.strip()
    except Exception:
        return ""

def extract_extra_info(driver):
    instrumental = "No"
    ai_generated = "No"
    try:
        info_container = driver.find_element(By.CSS_SELECTOR, "div.px-8.py-2.bg-gray-light.flex.flex-col.divide-y.divide-gray")
        info_divs = info_container.find_elements(By.CSS_SELECTOR, "div.grid.grid-cols-1.md\\:grid-cons-8.py-6")
        for div in info_divs:
            label = div.find_element(By.CSS_SELECTOR, "span.font-\\[500\\].md\\:col-span-2").text.strip()
            value = div.find_element(By.CSS_SELECTOR, "span.md\\:col-span-6").text.strip()
            if "Instrumental" in label:
                instrumental = value
            if "AI generated?" in label:
                ai_generated = value
    except Exception:
        pass
    return instrumental, ai_generated

def download_audio_file(file_url, filepath):
    response = requests.get(file_url, stream=True)
    if response.status_code == 200:
        with open(filepath, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
    else:
        print(f"Error downloading file from {file_url}; status: {response.status_code}")

def process_audio_page(audio_url, driver, index):
    driver.get(audio_url)

    track_info = extract_track_info(driver)
    file_url = track_info.get("fileUrl", "")
    audio_name = track_info.get("title", "").strip()
    author = track_info.get("artistName", "").strip()

    genres = extract_genres(driver)
    duration = extract_duration(driver)
    instrumental, ai_generated = extract_extra_info(driver)

    metadata = {
        "audioName": audio_name,
        "author": author,
        "genres": genres,
        "instrumental": instrumental,
        "ai_generated": ai_generated,
        "duration": duration,
        "audio_url": audio_url
    }

    audio_filename = f"audio_{index:04d}.mp3"
    meta_filename = f"audio_{index:04d}.json"
    audio_filepath = os.path.join("crawled_data", "audio", audio_filename)
    meta_filepath = os.path.join("crawled_data", meta_filename)

    download_audio_file(file_url, audio_filepath)

    with open(meta_filepath, "w", encoding="utf-8") as f:
        json.dump(metadata, f, ensure_ascii=False, indent=4)

    return metadata

In [ ]:
def loop_over_menu_pages(base_url, total_pages, driver):
    all_links = []
    for page in tqdm(range(1, total_pages + 1), desc="Extracting Links", unit="page"):
        page_url = f"{base_url}?page={page}"
        try:
            links = extract_audio_links_from_menu(page_url, driver)
            all_links.extend(links)
        except Exception as e:
            print(f"Error on page {page}: {e}")
    return all_links

In [ ]:
os.makedirs("crawled_data", exist_ok=True)
os.makedirs(os.path.join("crawled_data", "audio"), exist_ok=True)

base_url = "https://freemusicarchive.org/genre/piano/"
total_pages = 50
sample_idx = 1
audio_links = loop_over_menu_pages(base_url, total_pages, driver)
print(f"Total audio links extracted: {len(audio_links)}")

for audio_url in tqdm(audio_links, desc="Downloading Audio and Metadata", unit="audio"):
    try:
        process_audio_page(audio_url, driver, sample_idx)
        sample_idx += 1
    except:
        print(f'Error at: {audio_url}')
        continue
    time.sleep(0.5)

driver.quit()

## **4. Save data to file**

In [ ]:
!zip -r crawled_piano_audio_50_pages.zip crawled_data